In [ ]:
import numpy as np
from scipy.integrate import quad

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import DataSet

ds = DataSet.from_hdf5_file('sgrb.h5')
data = ds.to_stan_dict()

In [ ]:
K = np.random.rand(data["N_intervals"]) * 0.5 + 1.5
ec = np.random.rand(data["N_intervals"]) * 50 + 175
alpha = np.random.rand(data["N_intervals"]) * 0.5 - 1.5
data.update(dict(K = K, ec = ec, alpha = alpha))

## Python

In [ ]:
def cpl(x, K, ec, alpha):
    return K * (x)**alpha * np.exp(-x / ec)

def integral_flux_int(ebounds_lo, ebounds_hi, K, ec, alpha):
    try:
        integral = np.zeros(ebounds_hi.shape)
        for i, (lo, hi) in enumerate(zip(ebounds_lo, ebounds_hi)):
            integral[i] = quad(cpl, lo, hi, (K, ec, alpha))[0]
    except:
        integral = quad(cpl, ebounds_lo, ebounds_hi, (K, ec, alpha))[0]
    return integral

def integral_flux_simpson(ebounds_lo, ebounds_hi, K, ec, alpha):
    ebounds_add = (ebounds_hi - ebounds_lo) / 6
    ebounds_half = (ebounds_hi + ebounds_lo) / 2
    return ebounds_add * (
        cpl(ebounds_lo, K, ec, alpha)
        + 4 * cpl(ebounds_half, K, ec, alpha)
        + cpl(ebounds_hi, K, ec, alpha)
    )

In [ ]:
print(integral_flux_int(100,105,K[0],ec[0],alpha[0]))
print(integral_flux_simpson(100,105,K[0],ec[0],alpha[0]))

In [ ]:
print((data["response"][1, 1, :data["N_chan"][1,1]]).shape)
print(integral_flux_int(data["ebounds_lo"][1, 1, :data["N_echan"][1, 1]], data["ebounds_hi"][1, 1, :data["N_echan"][1, 1]], K[1], ec[1], alpha[1]).shape)

In [ ]:
expected_model_counts_int = np.zeros((max(data["N_dets"]), data["max_n_chan"]))

for i in range(data["N_intervals"]):
    for j in range(data["N_dets"][i]):
        expected_model_counts_int[j, :data["N_chan"][i,j]] = (
            data["response"][i, j, :data["N_chan"][i,j], :data["N_echan"][i,j]] @ integral_flux_int(
                    data["ebounds_lo"][i, j, :data["N_echan"][i, j]],
                    data["ebounds_hi"][i, j, :data["N_echan"][i, j]],
                    K[i],
                    ec[i],
                    alpha[i]
            )
        ) * data["exposure"][i,j]

expected_model_counts_simpson = np.zeros((max(data["N_dets"]), data["max_n_chan"]))

for i in range(data["N_intervals"]):
    for j in range(data["N_dets"][i]):
        expected_model_counts_simpson[j, :data["N_chan"][i,j]] = (
            data["response"][i, j, :data["N_chan"][i,j], :data["N_echan"][i,j]] @ integral_flux_simpson(
                    data["ebounds_lo"][i, j, :data["N_echan"][i, j]],
                    data["ebounds_hi"][i, j, :data["N_echan"][i, j]],
                    K[i],
                    ec[i],
                    alpha[i]
            )
        ) * data["exposure"][i,j]
expected_model_counts_simpson

## Stan

In [ ]:
mt = get_model("model_test")
mt.build_model()
mt.model.sample(data=data, fixed_param=True, show_console=True)

In [ ]:
expected_model_counts_stan = [[2706.97,358.426,242.647,167.926,118.496,85.0209,62.0171,45.7495,33.9125,25.4515,19.3539,14.8111,14.549,10.2769,7.35048,5.30416,3.84576,2.80905,2.05612,1.50961,1.11533,0.8275,0.614568,0.551521,0.383126,0.267733,0.188038,0.131845,0.0917454,0.0637916,0.044307,0.0354618,0.0226121,0.0143822,0.0092011,0.00593289,0.00385676,0.00250522,0.00186671,0.00119297,0.000790832,0.000538182,0.000368203,0.000250391,0.000188759,0.000119363,7.53694e-05,4.7437e-05,2.97669e-05,1.86208e-05,1.27296e-05,7.42759e-06,4.29811e-06,2.4746e-06,1.53238e-06,8.29377e-07,4.51886e-07,2.44602e-07,1.40566e-07,7.06309e-08,3.54915e-08,1.79363e-08,9.65341e-09,4.63378e-09,2.23123e-09,1.13692e-09,5.18364e-10,2.36192e-10,1.12493e-10,4.76913e-11,2.13651e-11,8.58777e-12,3.49428e-12,1.47514e-12,5.72756e-13,2.29274e-13,8.40436e-14,3.16258e-14,1.12045e-14,3.63385e-15,1.19778e-15,3.70638e-16,1.16479e-16,3.51371e-17,9.73155e-18,2.63167e-18,7.05597e-19,1.81126e-19,4.36463e-20,9.70498e-21,2.08728e-21,3.98194e-22,8.04227e-23,1.39978e-23,2.35533e-24,3.66007e-25,4.6777e-26,6.52107e-27,7.9352e-28,8.4267e-29,7.51565e-30,6.31385e-31,5.28528e-32,3.45222e-33,1.89843e-34,1.18831e-35,4.89038e-37,1.88122e-38,6.0548e-40,1.49689e-41,3.7557e-43,5.34523e-45,7.78865e-47,7.29765e-49,6.00626e-51,4.27945e-53,1.28535e-55,3.43887e-58,7.11984e-61,7.74991e-64,5.59112e-67,4.3998e-70,2.41377e-73,4.4187e-77,4.6446e-81,2.22754e-85,2.85289e-90,1.41438e-97],[54.5641,64.4593,67.669,77.7125,107.373,148.359,192.349,280.941,338.484,390.4,431.406,456.615,471.534,478.256,554.839,542.307,530.422,516.171,497.747,478.33,530.73,506.979,469.626,344.729,254.25,270.437,257.389,248.719,242.437,260.959,251.215,241.126,232.653,247.895,239.534,230.932,222.44,233.913,224.745,214.793,221.123,210.057,199.637,203.598,191.46,179.328,179.273,166.771,156.14,155.997,145.802,136.129,135.205,126.533,125.574,118.035,116.939,109.173,106.505,98.7643,91.5787,89.0726,82.2781,79.4251,76.3241,69.9723,66.6565,60.584,57.2263,51.8514,48.9829,46.1862,41.8026,39.03,36.1675,32.3014,29.8855,27.5862,25.3002,22.4278,20.5317,18.7607,17.0634,15.0421,13.6312,12.329,11.1331,10.0399,9.0112,8.06574,7.20615,6.40264,5.67736,5.06127,4.50934,4.00038,3.54481,3.19837,2.82001,2.47793,2.16237,1.88402,1.67467,1.46777,1.28343,1.13653,0.986186,0.867389,0.74962,0.656497,0.564112,0.488236,0.420716,0.356886,0.307243,0.265014,0.227352,0.194373,0.163524,0.13877,0.117232,0.098954,0.0837043,0.0716478,0.0608917,0.0728874,0.0563248,0.172851],[83.1957,118.317,154.518,207.169,294.803,407.189,507.89,717.006,807.227,894.23,938.313,962.621,968.286,959.885,1080.03,1020.11,966.822,922.37,879.056,837.59,918.476,880.065,832.418,667.243,442.192,467.002,446.002,429.163,413.029,440.212,421.672,404.253,388.092,409.391,390.105,371.017,353.687,367.785,349.571,331.257,338.645,318.636,299.177,301.92,282.528,263.957,263.611,245.44,229.768,229.434,215.018,201.125,199.354,185.493,182.537,169.687,166.334,154.121,150.028,138.774,128.17,123.725,113.126,107.886,102.214,92.5303,87.3359,78.8268,73.8673,66.1786,61.5595,57.0616,50.9605,47.1576,43.3868,38.4197,35.1555,32.0888,29.1719,25.6693,23.342,21.198,19.2104,16.8777,15.2016,13.6464,12.2339,10.9856,9.84135,8.77153,7.77077,6.87727,6.10288,5.43261,4.84622,4.31411,3.8364,3.44716,3.01217,2.6282,2.29903,2.01732,1.79661,1.56685,1.36536,1.2115,1.05845,0.936602,0.808638,0.703718,0.601037,0.520966,0.449453,0.380615,0.327064,0.281827,0.243745,0.210546,0.178101,0.151186,0.127539,0.107826,0.0912701,0.0782549,0.0663215,0.0787965,0.0601367,0.182063],[63.2667,78.9547,86.8589,102.31,145.753,207.259,268.842,396.529,474.988,541.754,594.483,629.788,644.327,646.193,739.478,705.382,671.916,640.198,613.242,593.211,656.187,629.203,594.826,471.099,313.406,331.663,318.631,309.93,302.677,327.123,316.692,305.296,293.153,309.262,296.975,285.61,274.607,286.474,272.455,259.049,265.565,250.471,235.975,238.36,222.389,207.138,206.22,191.356,177.919,176.886,165.148,154.625,153.604,143.575,141.906,132.477,130.348,121.307,118.85,110.679,102.937,99.9067,91.9421,88.1306,84.0662,76.6687,72.9171,66.3125,62.7436,56.8017,53.3176,49.7907,44.7634,41.7347,38.623,34.3167,31.4644,28.7713,26.232,23.2276,21.3037,19.5182,17.7589,15.6223,14.1007,12.7286,11.4597,10.2893,9.20098,8.23615,7.38159,6.5951,5.86963,5.2241,4.62725,4.07454,3.57925,3.22462,2.84646,2.51042,2.19163,1.90391,1.67897,1.45771,1.27122,1.12537,0.977129,0.857462,0.738039,0.642369,0.548577,0.472604,0.404834,0.341517,0.292967,0.250939,0.214496,0.183165,0.154289,0.131018,0.110558,0.0932549,0.0784317,0.0665698,0.0557016,0.0653464,0.0496853,0.148177]]

In [ ]:
for i,j in zip(expected_model_counts_stan, expected_model_counts_simpson):
    for k,l in zip(i,j):
        print(abs(k - l) < 0.001)